<a href="https://colab.research.google.com/github/saikrishna1108/Computer_Architecture/blob/main/Homework6_1Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 1 (40 pts):

a. Develop a Fully Connected Neural Network with only one hidden layer (size of 32) to predict the housing value for the housing dataset. Make sure to include all input features. Compare your training loss value and validation results against the linear regression you implemented in Homework 5. Can you compare your model complexity (number of trainable parameters) against linear regression? Note: Perform 20%, and 80% split for training and validation.



b. We will increase the network complexity by adding two additional hidden layers, the hidden layers overall. My suggestions for the size of layers are: 32, 64, 16, respectively. Please redesign the network and compare your training loss value and validation results against the linear regression you implemented in Homework 5 and Problem 1.a. Can you compare your model complexity? Note: Use the same 20%, and 80% split for training and validation.




In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [91]:
Housing_data = pd.read_csv("/content/drive/MyDrive/Housing.csv")
Housing_data = Housing_data.replace({'yes':1,'no':0})
Housing_data['furnishingstatus'] =Housing_data['furnishingstatus'].map({'furnished':2,'semi-furnished':1,'unfurnished':0})
Housing_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [197]:
X = Housing_data.iloc[:,1:]
m = X.shape[0]
Y = np.array(Housing_data['price'])
Y = np.resize(Y,(len(Y),1))

In [199]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(545, 12)

In [200]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100)

Y_train = torch.from_numpy(Y_train).to(torch.float32)
X_train = torch.from_numpy(X_train).to(torch.float32)
Y_test = torch.from_numpy(Y_test).to(torch.float32)
X_test = torch.from_numpy(X_test).to(torch.float32)

In [207]:
import torch.nn as nn
model=nn.Sequential(nn.Linear(12,32),
                        nn.Tanh(),
                        nn.Linear(32,1))
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [202]:
def new_loss(pred,actual):
  mse = torch.mean((pred - actual) ** 2)
  rmse = torch.sqrt(mse)
  return rmse


In [203]:
def training_loop(n_epochs, optimizer, model, loss_fn, X_train, X_test, Y_train, Y_test):
  plotted_loss = np.zeros(11)
  count = 0
  for epoch in range(1, n_epochs + 1):
    Y_pred_train = model(X_train)
    train_loss = loss_fn(Y_pred_train, Y_train)

    with torch.no_grad(): # <1>
      Y_pred_test = model(X_test)
      test_loss = loss_fn(Y_pred_test, Y_test)
      assert test_loss.requires_grad == False # <2>

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    if ((epoch == 1) or (epoch % 500 == 0)):
      plotted_loss[count] = test_loss
      count += 1
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}, Validation loss {test_loss.item():.4f}")

  return plotted_loss

In [208]:
single_seq = training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = model,
    loss_fn = new_loss,
    X_train = X_train,
    X_test = X_test,
    Y_train = Y_train,
    Y_test = Y_test)

Epoch 1, Training loss 5145639.0000, Validation loss 5015851.5000
Epoch 500, Training loss 5145474.5000, Validation loss 5015687.5000
Epoch 1000, Training loss 5145318.5000, Validation loss 5015532.5000
Epoch 1500, Training loss 5145164.0000, Validation loss 5015378.5000
Epoch 2000, Training loss 5145009.5000, Validation loss 5015225.0000
Epoch 2500, Training loss 5144855.5000, Validation loss 5015072.0000
Epoch 3000, Training loss 5144701.0000, Validation loss 5014918.5000
Epoch 3500, Training loss 5144547.5000, Validation loss 5014765.5000
Epoch 4000, Training loss 5144394.0000, Validation loss 5014613.0000
Epoch 4500, Training loss 5144240.0000, Validation loss 5014459.5000
Epoch 5000, Training loss 5144086.0000, Validation loss 5014306.5000


In [209]:
multi_model = nn.Sequential(
  nn.Linear(12, 32),
  nn.Tanh(),
  nn.Linear(32, 64),
  nn.Tanh(),
  nn.Linear(64, 16),
  nn.Tanh(),
  nn.Linear(16, 1)
)
multi_optimizer = optim.Adam(multi_model.parameters(), lr=0.01)

In [210]:
multi_seq = training_loop(
    n_epochs = 5000,
    optimizer = multi_optimizer,
    model = multi_model,
    loss_fn = new_loss,
    X_train = X_train,
    X_test = X_test,
    Y_train = Y_train,
    Y_test = Y_test)

Epoch 1, Training loss 5145639.0000, Validation loss 5015851.0000
Epoch 500, Training loss 5145557.5000, Validation loss 5015770.5000
Epoch 1000, Training loss 5145478.0000, Validation loss 5015691.0000
Epoch 1500, Training loss 5145398.5000, Validation loss 5015612.5000
Epoch 2000, Training loss 5145319.5000, Validation loss 5015533.5000
Epoch 2500, Training loss 5145240.0000, Validation loss 5015455.0000
Epoch 3000, Training loss 5145161.0000, Validation loss 5015376.0000
Epoch 3500, Training loss 5145082.0000, Validation loss 5015297.0000
Epoch 4000, Training loss 5145003.0000, Validation loss 5015218.5000
Epoch 4500, Training loss 5144923.5000, Validation loss 5015139.5000
Epoch 5000, Training loss 5144844.0000, Validation loss 5015061.0000
